# PokeAPI Final Project

In [ ]:
# Importing all essential libraries. More to be added as project continues
import pandas as pd
import scipy.stats as stats 
import json
import requests

ModuleNotFoundError: No module named 'scipy'

In [ ]:
# Function to fetch a Pokemon's data
def fetch_pokemon_data(pokemon):
    # Extract the pokemon's data
    pokemon_url = pokemon["url"]
    pokemon_resp = requests.get(pokemon_url)
    pokemon_data = pokemon_resp.json()

    # Extracting the pokemon's name, type, and stats from pokemon_data
    name = pokemon_data["name"]
    types = [type['type']['name'] for type in pokemon_data["types"]] # get all types if a pokemon has multiple types
    stats = {stat["stat"]["name"]: stat["base_stat"] for stat in pokemon_data["stats"]} # get all stats as a dictionary (stat["stat"]["name"]: stat["base_stat"] creates a key-value pair in which the name is the key and the base stat is the value)

    # Extracting the pokemon's generation number
    species_url = pokemon_data["species"]["url"]
    species_resp = requests.get(species_url)
    species_data = species_resp.json()
    generation = species_data["generation"]["name"] 

    generation_num = generation.split("-")[-1] # retrieves the last elem of the list generated by split (the roman numeral)
    generation_roman_lst = ["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix"]

    if generation_num in generation_roman_lst:
        index = generation_roman_lst.index(generation_num)
        generation_num = int(generation_num.replace(generation_roman_lst[index], str(index + 1)))

    # Add Pokemon's name and types into the stats dictionary (ensuring they're there if not)
    stats["name"] = name
    stats["types"] = types
    stats["generation"] = generation_num

    return stats # returns as a dictionary
    

In [ ]:
# Fetching the API (we are only looking at the first 493 Pokemon, Gens 1-4)
api_url = "https://pokeapi.co/api/v2/pokemon?limit=493&offset=0"
response = requests.get(api_url)
pokeapi_data = response.json()

# Creating a list which will store all the Pokemons' information
pokemon_list = []

# Loop which will fetch each Pokemons' data and append it to a list
for pokemon in pokeapi_data["results"]:
    pokemon_entry = fetch_pokemon_data(pokemon)
    pokemon_list.append(pokemon_entry)

# Convert the list to a DataFrame
pokemon_df = pd.DataFrame(pokemon_list)
pokemon_df = pokemon_df[["name", "types", "generation", "hp", "attack", "defense", "special-attack", "special-defense", "speed"]]
pokemon_df

,name,types,generation,hp,attack,defense,special-attack,special-defense,speed
0,bulbasaur,"[grass, poison]",1,45,49,49,65,65,45
1,ivysaur,"[grass, poison]",1,60,62,63,80,80,60
2,venusaur,"[grass, poison]",1,80,82,83,100,100,80
3,charmander,[fire],1,39,52,43,60,50,65
4,charmeleon,[fire],1,58,64,58,80,65,80
...,...,...,...,...,...,...,...,...,...
488,phione,[water],4,80,80,80,80,80,80
489,manaphy,[water],4,100,100,100,100,100,100
490,darkrai,[dark],4,70,90,90,135,90,125
491,shaymin-land,[grass],4,100,100,100,100,100,100


In [ ]:
# Describes the Pokemon DataFrame
pokemon_df.describe()

,generation,hp,attack,defense,special-attack,special-defense,speed
count,493.000000,493.000000,493.000000,493.000000,493.000000,493.000000,493.000000
mean,2.401623,67.912779,73.608519,70.231237,68.133874,69.259635,65.561866
std,1.135602,27.613228,29.209693,30.682469,28.573296,27.883350,27.340646
min,1.000000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000
25%,1.000000,50.000000,50.000000,50.000000,45.000000,50.000000,45.000000
50%,2.000000,65.000000,72.000000,65.000000,65.000000,65.000000,65.000000
75%,3.000000,80.000000,92.000000,85.000000,90.000000,85.000000,85.000000
max,4.000000,255.000000,165.000000,230.000000,154.000000,230.000000,160.000000


In [ ]:
# Displaying the count to check for missing values
display(pokemon_df.count())
display(pokeapi_data["count"])

name               493
types              493
generation         493
hp                 493
attack             493
defense            493
special-attack     493
special-defense    493
speed              493
dtype: int64

1302

We want to see if there is a relationship between the generation number and number of Psychic-type Pokémon (including double types) released in the dataset. To do so, we perform a chi-squared test for independence.  

In [ ]:
# extract all Psychic-type Pokemon from the dataset
psychic_types = pokemon_df[pokemon_df['types'].apply(lambda x: 'psychic' in x)]
psychic_types
# perform test

,name,types,generation,hp,attack,defense,special-attack,special-defense,speed
62,abra,[psychic],1,25,20,15,105,55,90
63,kadabra,[psychic],1,40,35,30,120,70,105
64,alakazam,[psychic],1,55,50,45,135,95,120
78,slowpoke,"[water, psychic]",1,90,65,65,40,40,15
79,slowbro,"[water, psychic]",1,95,75,110,100,80,30
95,drowzee,[psychic],1,60,48,45,43,90,42
96,hypno,[psychic],1,85,73,70,73,115,67
101,exeggcute,"[grass, psychic]",1,60,40,80,60,45,40
102,exeggutor,"[grass, psychic]",1,95,95,85,125,75,55
120,starmie,"[water, psychic]",1,60,75,85,100,85,115
